<a href="https://colab.research.google.com/github/simaoXVS/MPC-MLF/blob/main/MPA_MLF_Final_SimaoXavier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MPA-MLF Final Project
###Simão Xavier 263873

# Importing

In [ ]:
!pip install timm

In [ ]:
!pip install wandb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil

import time
import random # for torch seed
import os # for torch seed
from pathlib import Path

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW, RMSprop # optmizers
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau # Learning rate schedulers

import albumentations as A

import timm
import wandb

from torchvision import datasets, transforms
from torchvision import models
from tqdm import tqdm
from skimage.color import rgba2rgb
from skimage import data
from PIL import Image

In [ ]:
print('timm version', timm.__version__)
print('torch version', torch.__version__)

# Read

In [ ]:
!rm -rf train_data_unlabeled/

In [ ]:
!unzip train_data_unlabeled.zip

In [ ]:
!unzip test_data_unlabeled.zip

In [ ]:
train_data_path = Path("train_data_unlabeled/")
test_data_path = Path("test_data_unlabeled/")
train_labels_csv = "y_train.csv"

In [ ]:
train_images = list(train_data_path.iterdir())
test_images = list(test_data_path.iterdir())
pd.read_csv(train_labels_csv)

In [ ]:
plt.imshow(plt.imread(train_images[0]))

In [ ]:
# Paths
root_dir = '.'  # Update this path
dataset_dir = os.path.join(root_dir, 'dataset')
images_dir = os.path.join(root_dir, 'train_data_unlabeled')
labels_file = os.path.join(root_dir, 'y_train.csv')

# CFG

This class enables easy configuration to simulate and understand results.

You can tweak some parameters and see how they impact on the metrics, plots, and predictions.

In [ ]:
class CFG:
  DEBUG = False # True False

  ### input: not configurable
  IMG_HEIGHT = 45
  IMG_WIDTH = 51
  N_CLASS = 4

  ### split train and validation sets
  split_fraction = 1.0

  ### model
  # model_name = "resnet34"
  model_name = "efficientnet_b1_pruned"
  pretrained = True

  ### training
  print_freq = 100
  BATCH_SIZE = 32
  N_EPOCHS = 3 if DEBUG else 80

  ### set only one to True
  save_best_loss = False
  save_best_accuracy = True

  ### optimizer
#   optimizer = 'adam'
  optimizer = 'adamw'
  # optimizer = 'rmsprop'

  LEARNING_RATE = 1e-3

  weight_decay = 0.1 # for adamw
  l2_penalty = 0.01 # for RMSprop
  rms_momentum = 0 # for RMSprop

  ### learning rate scheduler (LRS)
  scheduler = 'ReduceLROnPlateau'
#   scheduler = 'CosineAnnealingLR'
  plateau_factor = 0.5
  plateau_patience = 3
  cosine_T_max = 4
  cosine_eta_min = 1e-8
  verbose = True

  ### processing
  resize = True
  resize_value = (64, 64)
  padding = True

  ### augmentations
  probability = 0.6
  random_horizontal_flip = True
  random_vertical_flip = True

  random_seed = 88

cfg_dict = dict(CFG.__dict__)
wandb_dict = dict()
for key, value in cfg_dict.items():
    if not key.startswith('__'):
      wandb_dict.update({key: value})

In [ ]:
# Initialize a new run
username = "simaoxavier"
wandb.init(project='DopplerClassifier', entity=username, config=wandb_dict)

In [ ]:
# detect and define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
# for reproducibility
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed = CFG.random_seed)

# Prepare Data

In [ ]:
!rm -rf dataset

In [ ]:
# Load labels
df = pd.read_csv(labels_file)

# Split data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

# Function to create directories and move files
def prepare_data(df, dest_folder):
    dest_dir = os.path.join(dataset_dir, dest_folder)
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for _, row in df.iterrows():
        file_path = os.path.join(images_dir, f"img_{str(row['id']+1)}.png")
        if not os.path.exists(file_path):
          print(f"img_{row['id']+1}.png not found in {images_dir}!")
          continue
        label_dir = os.path.join(dest_dir, str(row['target']))
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)
        shutil.copy(file_path, os.path.join(label_dir, f"img_{str(row['id']+1)}.png"))

# Submission run
if CFG.split_fraction == 1.0:
  prepare_data(df, 'train')
  prepare_data(val_df, 'val')
# Prepare train and validation directories
else:
  prepare_data(train_df, 'train')
  prepare_data(val_df, 'val')

print("Data has been split and placed into train and val folders.")


In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_transform(target_size=(64, 64), thresh=0.4, mean=0.0, std=0.1, with_augmentations=False, probability=CFG.probability):
    def remove_alpha_channel(img):
        # Convert RGBA to RGB
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        return img

    def pad_if_needed(img):
        width, height = img.size
        max_side = max(width, height)
        padding = (0, 0, max_side - width, max_side - height)  # left, top, right, bottom
        return transforms.functional.pad(img, padding, padding_mode='constant', fill=0)

    def to_binary(img):
        # Convert image to grayscale
        gray_img = img.convert('L')
        # Apply threshold to convert image to binary
        return gray_img.point(lambda x: 255 if x > thresh * 255 else 0).convert('RGB')

    if with_augmentations:
        transform = transforms.Compose([
            transforms.Lambda(remove_alpha_channel),  # Remove alpha channel if present
            transforms.Lambda(pad_if_needed),         # Pad to make square
            transforms.Resize(target_size),           # Resize to the target size
            transforms.ToTensor(),                     # Convert to tensor
            transforms.RandomHorizontalFlip(probability),  # Random horizontal flip
            transforms.RandomVerticalFlip(probability),  # Random horizontal flip
        ])
    else:
        transform = transforms.Compose([
            transforms.Lambda(remove_alpha_channel),  # Remove alpha channel if present
            transforms.Lambda(pad_if_needed),         # Pad to make square
            transforms.Resize(target_size),           # Resize to the target size
            transforms.ToTensor(),                    # Convert to tensor
        ])
    return transform

# Transforms
train_transform = get_transform(with_augmentations=True)
valid_transform = get_transform()

# Load datasets
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_dir, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_dir, 'val'), transform=valid_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=2)

###############
img = Image.open('dataset/train/3/img_10005.png')  # Make sure to load an actual file path

# Get the transform function
debug_transform = get_transform(with_augmentations=True, probability=1.0)

# Apply the transform
transformed_img = debug_transform(img)

# If you want to visualize the result using matplotlib
plt.imshow(np.transpose(transformed_img.numpy(), (1, 2, 0)))
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()

# Transfer Learning: timm

- [timm.create_model and timm.list_models](https://huggingface.co/docs/timm/reference/models)
- [timm docs](https://huggingface.co/docs/timm/feature_extraction)

In [ ]:
class DopplerModel(nn.Module):
    def __init__(self, model_name = CFG.model_name, pretrained = CFG.pretrained):
        super().__init__()

        self.model_name = model_name
        self.cnn = timm.create_model(self.model_name, pretrained = pretrained, num_classes = CFG.N_CLASS)

    def forward(self, x):
        x = self.cnn(x)
        return x

In [ ]:
CFG.model_name

# Optimizer

[torch optimizer documentation](https://pytorch.org/docs/stable/optim.html#)

Function to get the optimizer to be used (can be tuned in CFG class).

In [ ]:
def get_optimizer(lr = CFG.LEARNING_RATE):

  if CFG.optimizer == 'adam':
      optimizer = Adam(model.parameters(), lr=lr, weight_decay = CFG.weight_decay, amsgrad = False)

  elif CFG.optimizer == 'adamw':
      optimizer = AdamW(model.parameters(), lr = lr, weight_decay = CFG.weight_decay)

  elif CFG.optimizer == 'rmsprop':
      optimizer = RMSprop(model.parameters(), lr = lr, weight_decay = CFG.l2_penalty, momentum = CFG.rms_momentum)

  else:
      print('Optimizer is not defined')

  return optimizer

# LR Scheduler

[torch LRS documentation](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

Function to get the Learning Rate Scheduler to be used (can be tuned in CFG class).

`torch.optim.lr_scheduler` provides several methods to adjust the learning rate based on the number of epochs.

`torch.optim.lr_scheduler.ReduceLROnPlateau` allows dynamic learning rate reducing based on some validation measurements.

Learning rate scheduling should be applied after optimizer’s update.

In [ ]:
def get_scheduler(optimizer):

  if CFG.scheduler=='ReduceLROnPlateau':
      scheduler = ReduceLROnPlateau(optimizer, mode='max', factor = CFG.plateau_factor, patience = CFG.plateau_patience, verbose = CFG.verbose)

  elif CFG.scheduler=='CosineAnnealingLR':
      scheduler = CosineAnnealingLR(optimizer, T_max = CFG.cosine_T_max, eta_min = CFG.cosine_eta_min)

  else:
      print('LR Scheduler is not defined')

  return scheduler

# Train function

In [ ]:
def train_fn(train_loader, model, criterion, optmizer, device):
  # switch to train mode
  model.train()


  size = len(train_loader.dataset)
  num_batches = len(train_loader)

  loss, correct = 0, 0

  ################################# train #################################

  for batch, (X, y) in enumerate(train_loader):

    start = time.time()

    device = torch.device(device)
    X, y = X.to(device), y.to(device)

    # compute predictions and loss
    optimizer.zero_grad()
    pred = model(X)
    loss = criterion(pred, y.long().squeeze())
    current = batch * len(X)

    # Backpropagation: only in train function, not done in validation function
    loss.backward()
    optimizer.step()

    # sum correct predictions
    y_pred, y_true = torch.argmax(pred, axis=1), y.long().squeeze()
    correct += (y_pred == y_true).type(torch.float).sum().item()

    end = time.time()
    time_delta = np.round(end - start, 3)

    # log
    loss, current = np.round(loss.item(), 5), batch * len(X)

  # metrics: calculate accuracy and loss for epoch (all batches)
  correct /= size # epoch accuracy
  loss /= num_batches # epoch loss

  print(f"Train: Accuracy: {(100*correct):>0.2f}%, Avg loss: {loss:>5f} \n")

  return loss, correct

# Validation function

[torch.no_grad documentation](https://pytorch.org/docs/stable/generated/torch.no_grad.html)

Use `torch.no_grad` when you are sure you will not call `Tensor.backward()`. It reduces memory and time consumption.

In [ ]:
def valid_fn(valid_loader, model, criterion, device):
  model.eval()

  size = len(valid_loader.dataset)
  num_batches = len(valid_loader)

  loss, correct = 0, 0

  ################################# validation #################################

  with torch.no_grad(): # disable gradients
    for batch, (X, y) in enumerate(valid_loader):

      start = time.time()

      device = torch.device(device)
      X, y = X.to(device), y.to(device)

      # compute predictions and loss
      pred = model(X)
      loss = criterion(pred, y.long().squeeze())
      current = batch * len(X)

      # sum correct predictions
      y_pred, y_true = torch.argmax(pred, axis=1), y.long().squeeze()
      correct += (y_pred == y_true).type(torch.float).sum().item()

      end = time.time()
      time_delta = np.round(end - start, 3)

      # log
      loss, current = np.round(loss.item(), 5), batch * len(X)

  # metrics: calculate accuracy and loss for epoch (all batches)
  correct /= size # epoch accuracy
  loss /= num_batches # epoch loss

  print(f"Valid: Accuracy: {(100*correct):>0.2f}%, Avg loss: {loss:>5f} \n")

  return loss, correct

# Run training

In [ ]:
start = time.time()

# define loss function
loss_fn = nn.CrossEntropyLoss()

# instantiate model
device = torch.device(device)
model = DopplerModel().to(device) # move the model to GPU before constructing optimizers for it

print('\n ******************************* Using backbone: ', CFG.model_name, " ******************************* \n")

# define optimizer
optimizer = get_optimizer(lr = CFG.LEARNING_RATE)

# define scheduler
scheduler = get_scheduler(optimizer)

OUTPUT_PATH = './'

train_dataset = train_dataset
valid_dataset = val_dataset
train_dataloader = train_loader
valid_dataloader = val_loader


train_loss_history = []
train_acc_history = []
valid_loss_history = []
valid_acc_history = []
LR_history = []

best_loss = np.inf
best_epoch_loss = 0
best_acc = 0
best_epoch_acc = 0

print('Starting Training...\n')

start_train_time = time.time()

for epoch in range(0, CFG.N_EPOCHS):
  print(f"\n-------------------------------   Epoch {epoch + 1}   -------------------------------\n")
  start_epoch_time = time.time()

  # train
  train_loss, train_acc = train_fn(train_dataloader, model, loss_fn, optimizer, device)
  train_loss_history.append(train_loss)
  train_acc_history.append(train_acc)

  # validation
  valid_loss, valid_acc = valid_fn(valid_dataloader, model, loss_fn, device)
  valid_loss_history.append(valid_loss)
  valid_acc_history.append(valid_acc)

  # apply LR scheduler after each epoch
  if isinstance(scheduler, ReduceLROnPlateau):
      scheduler.step(valid_loss)

  elif isinstance(scheduler, CosineAnnealingLR):
      scheduler.step()

  # save LR value to plot later
  for param_group in optimizer.param_groups:
    LR_history.append(param_group['lr'])

  # Log metrics
  wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": valid_loss, "val_accuracy": valid_acc, "lr": LR_history[-1],"epoch": epoch})

  # save validation loss if it was improved (reduced)
  if valid_loss < best_loss:
    best_epoch_loss = epoch + 1
    best_loss = valid_loss
    if CFG.save_best_loss:
      # save the model's weights and biases only if CFG.save_best_loss == True
      torch.save(model.state_dict(), OUTPUT_PATH + f"DopplerModel_ep{best_epoch_loss}.pth")

  # save validation accuracy if it was improved (increased)
  if valid_acc > best_acc:
    best_epoch_acc = epoch + 1
    best_acc = valid_acc
    if CFG.save_best_accuracy:
      # save the model's weights and biases only if CFG.save_best_accuracy == True
      torch.save(model.state_dict(), OUTPUT_PATH + f"DopplerModel_ep{best_epoch_acc}.pth")
      wandb.save(f"DopplerModel_ep{best_epoch_acc}.pth")

  # Save all epochs
  # torch.save(model.state_dict(), OUTPUT_PATH + f"DopplerModel_ep{epoch + 1}.pth")
  # wandb.save(f"DopplerModel_ep{epoch + 1}.pth")

  end_epoch_time = time.time()
  time_delta = np.round(end_epoch_time - start_epoch_time, 3)
  print("\n\nEpoch Elapsed Time: {} s".format(time_delta))

end_train_time = time.time()
print("\n\nTotal Elapsed Time: {} min".format(np.round((end_train_time - start_train_time)/60, 3)))
print("Done!")

wandb.finish()

# Plot Epochs

Plot Train and Validation Loss and Accuracy for each Epoch.

In [ ]:
print('Best validation loss: ', round(best_loss, 5))
print('Best epoch (loss criteria in validation): ', best_epoch_loss)
print('\n')
print('Best validation accuracy: ', round(best_acc, 5))
print('Best epoch (accuracy criteria in validation): ', best_epoch_acc)

In [ ]:
fig = plt.figure(figsize = (18, 8))
fig.suptitle('Epoch Results', fontsize = 18)

abscissa = np.arange(1, CFG.N_EPOCHS + 1, 1)

# x_ticks according to CFG.N_EPOCHS for better visuailzation
if CFG.N_EPOCHS <= 20:
  x_ticks = np.arange(1, CFG.N_EPOCHS + 1, 1)
else:
  x_ticks = np.arange(1, CFG.N_EPOCHS + 1, int(CFG.N_EPOCHS/20) + 1)

# Loss plot
ax1 = plt.subplot(1, 2, 1)
ax1.plot(abscissa, train_loss_history, label='Training', color = 'black')
ax1.plot(abscissa, valid_loss_history, label='Validation', color = 'red')
plt.xticks(x_ticks)
plt.axhline(0, linestyle = 'dashed', color = 'grey')
plt.axvline(best_epoch_loss, linestyle = 'dashed', color = 'blue', label = 'Best val loss: ep ' + str(best_epoch_loss))
plt.axvline(best_epoch_acc, linestyle = 'dashed', color = 'green', label = 'Best val acc: ep ' + str(best_epoch_acc))
plt.title("Loss")
ax1.legend(frameon=False);

# Accuracy plot
ax2 = plt.subplot(1, 2, 2)
ax2.plot(abscissa, train_acc_history, label='Training', color = 'black')
ax2.plot(abscissa, valid_acc_history, label='Validation', color = 'red')
plt.xticks(x_ticks)
plt.axhline(0.99, linestyle = 'dashed', color = 'grey')
plt.axvline(best_epoch_loss, linestyle = 'dashed', color = 'blue', label = 'Best val loss: ep ' + str(best_epoch_loss))
plt.axvline(best_epoch_acc, linestyle = 'dashed', color = 'green', label = 'Best val acc: ep ' + str(best_epoch_acc))
plt.title("Accuracy")
ax2.legend(frameon=False);

In [ ]:
fig = plt.figure(figsize = (14, 8))

abscissa = np.arange(1, CFG.N_EPOCHS + 1, 1)

# x_ticks according to CFG.N_EPOCHS for better visuailzation
if CFG.N_EPOCHS <= 20:
  x_ticks = np.arange(1, CFG.N_EPOCHS + 1, 1)
else:
  x_ticks = np.arange(1, CFG.N_EPOCHS + 1, int(CFG.N_EPOCHS/20) + 1)

# LR plot
plt.plot(abscissa, LR_history, label='LR', color = 'orange')
plt.xticks(x_ticks)
plt.axhline(CFG.LEARNING_RATE, linestyle = 'dashed', color = 'grey')
plt.axhline(0, linestyle = 'dashed', color = 'grey')
plt.axvline(best_epoch_loss, linestyle = 'dashed', color = 'blue', label = 'Best val loss: ep ' + str(best_epoch_loss))
plt.axvline(best_epoch_acc, linestyle = 'dashed', color = 'green', label = 'Best val acc: ep ' + str(best_epoch_acc))
plt.title(f"Learning Rate vs Epochs: {CFG.scheduler}", fontsize = 16, color = 'orange')
plt.legend(frameon=False);

# Inference

In [ ]:
!mkdir test

In [ ]:
!mkdir test/unlabeled

In [ ]:
test_list = list(Path("test_data_unlabeled/").iterdir())
for file in test_list:
  id = (file.stem).split('_')[-1]
  shutil.copy(str(file),f"test/unlabeled/{id.zfill(6)}.png")

list(Path("test/unlabeled/").iterdir())

In [ ]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=1)[:, None]

def inference(test_loader, model):
    model.eval()

    predictions = []

    size = len(test_loader.dataset)
    num_batches = len(test_loader)

    model = DopplerModel().to(device)

    if CFG.save_best_loss: # load model with best validation loss
      model.load_state_dict(torch.load(OUTPUT_PATH + f"DopplerModel_ep{best_epoch_loss}.pth"))
    else: # load model with best validation accuracy
      model.load_state_dict(torch.load(OUTPUT_PATH + f"DopplerModel_ep{best_epoch_acc}.pth"))

    # load model from specific epoch
    # model.load_state_dict(torch.load(OUTPUT_PATH + f"DopplerModel_ep60.pth"))

    # disable gradients for inference
    with torch.no_grad():
      for batch, (X, _) in enumerate(test_loader):

        ################################# inference #################################
        start = time.time()
        current = batch * len(X)

        X = X.to(device)

        # compute predictions
        pred = model(X)
        # softmax
        y_pred = softmax(pred.detach().cpu().numpy()) # convert tensor to numpy and apply softmax
        y_pred = np.argmax(y_pred, axis = 1) # take the indice of the max value (higher probability: predicted class)

        # store results
        predictions.append(y_pred)

        # log
        end = time.time()
        time_delta = np.round(end - start, 5)

    test_predictions = np.concatenate(predictions, axis = 0) # join sequence of arrays along axis 0
    return test_predictions

In [ ]:

# instantiate Inference Dataset class (create inference Dataset)
test_transform = get_transform()
inference_dataset = datasets.ImageFolder(root=os.path.join('test'), transform=test_transform)

# Data loaders
inference_dataloader = DataLoader(inference_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
%%time
# run inference
predictions = inference(inference_dataloader, model)
predictions

In [ ]:
len(predictions)

# Submission

In [ ]:
submission = pd.read_csv("submission_example.csv")
submission["target"] = predictions

submission.to_csv(OUTPUT_PATH + 'submission.csv', index = False)
submission.head()

# Check predictions

In [ ]:
# check some predictions

fig = plt.figure(figsize = (12, 12))
fig.suptitle('Visualizing Predictions', fontsize = 24)

# define a range of predictions to plot
begin = 130
end = begin + 20

test_list = list(Path("test/unlabeled/").iterdir())

for i in range(begin, end):
  img = plt.imread(test_list[i])

  plt.subplot(4, 5, i + 1 - begin) # 4 rows and 5 columns plot
  label = str(submission.loc[i, 'target'])
  plt.title("Predicted label: " + label, color="red") # write label in each image title
  plt.imshow(np.squeeze(img), cmap='gray') # plot image
  plt.axis('off')